# Plan for Day 2: Data Cleaning & Preprocessing

Based on the analysis above, here is our step-by-step plan for tomorrow. We will continue working in a new Jupyter Notebook to keep our process organized.

## Clean Column Names

The first step is always to standardize the column names. This makes them easier to work with in code.

**Task**: Convert all column names to lowercase and replace any spaces or special characters with underscores (_). For example, `Vehicle Images` will become `vehicle_images`.

## Remove Redundant and Unnecessary Columns

To make our dataset lighter and more focused, we will drop columns that don't add analytical value.

**Task**: Drop the `Time` column (as this information is already in the `Date` column) and the `vehicle_images` column.

## Handle Data Types

We need to ensure every column has the correct data type for analysis.

- **Task 1**: The `Date` column is already a `datetime64` object, which is perfect. We will rename it to `booking_timestamp` to be more descriptive.
- **Task 2**: Investigate the `Incomplete_Rides` column. It likely contains "Yes" and "No" strings. We will convert this into a more useful boolean format (True/False).

## Investigate and Handle Anomalies

We need to understand the data points that don't make logical sense.

**Task**: Filter the DataFrame to find all rows where `Ride_Distance` is 0. Examine their `Booking_Status`. This will help us confirm if they are, for example, instant cancellations, and decide if we should keep or remove them.

## Consolidate Cancellation Data

To make analyzing cancellations easier, we can combine the separate reason columns.

**Task**: Create a new column called `cancellation_reason`. This column will contain the reason from either the customer or driver column. If the ride was not cancelled, the value will be "Not Cancelled". This simplifies having to look in two separate columns. We can then drop the original two cancellation reason columns.

## Save the Cleaned Data

After all these steps, we will save our newly cleaned and processed DataFrame to a new file.

**Task**: Export the final, cleaned DataFrame to a CSV file named `ola_data_cleaned.csv`. This file will be the single source of truth for all our future analysis in SQL and Power BI.

### Day 2 Workflow Summary

| Step | Task | Expected Outcome |
|------|------|------------------|
| 1 | Standardize column names | All columns in lowercase with underscores |
| 2 | Remove redundant columns | Drop `Time` and `vehicle_images` |
| 3 | Fix data types | Convert `Incomplete_Rides` to boolean, rename `Date` |
| 4 | Handle anomalies | Investigate zero-distance rides |
| 5 | Consolidate cancellations | Single `cancellation_reason` column |
| 6 | Export clean data | Analysis-ready `ola_data_cleaned.csv` file |


In [2]:
import pandas as pd
import numpy as np

# --- 1. Load the Raw Dataset ---
# We start by loading the original Excel file again.
try:
    df = pd.read_excel(r'C:\Users\jaiku\PycharmProjects\Ola_Ride_Analytics\data\OLA_DataSet.xlsx')
    print("✅ Raw dataset loaded successfully for cleaning.")
except FileNotFoundError:
    print("❌ Error: 'ola_data.xlsx' not found. Make sure it's in the 'data' directory.")
    exit()

# --- 2. Clean Column Names ---
# Standardizing column names is a crucial first step for clean, readable code.
# We convert all names to lowercase and replace spaces with underscores.
print("\nOriginal column names:\n", df.columns)
df.columns = df.columns.str.lower().str.replace(' ', '_')
print("\nCleaned column names:\n", df.columns)


# --- 3. Remove Redundant and Unnecessary Columns ---
# We drop columns that don't add analytical value to simplify our DataFrame.
# 'time' is redundant because 'date' already contains the timestamp.
# 'vehicle_images' is not useful for our analysis.
columns_to_drop = ['time', 'vehicle_images']
df.drop(columns=columns_to_drop, inplace=True)
print(f"\n✅ Dropped unnecessary columns: {columns_to_drop}")


# --- 4. Handle Data Types and Rename Columns ---
# Correct data types are essential for accurate analysis.
# The 'date' column is already a datetime object, which is great.
# We'll rename it to be more descriptive.
df.rename(columns={'date': 'booking_timestamp'}, inplace=True)
print("\n✅ Renamed 'date' column to 'booking_timestamp'.")

# Let's inspect the 'incomplete_rides' column before converting it.
print("\nUnique values in 'incomplete_rides':", df['incomplete_rides'].unique())
# We'll convert 'Yes'/'No' to a boolean (True/False) for easier filtering.
# We use .loc to safely handle the mapping and avoid SettingWithCopyWarning.
df.loc[:, 'incomplete_rides'] = df['incomplete_rides'].map({'Yes': True, 'No': False})
print("✅ Converted 'incomplete_rides' to boolean type.")


# --- 5. Investigate and Handle Anomalies (Ride Distance = 0) ---
# A ride distance of 0 is a data anomaly. Let's investigate these records.
zero_dist_rides = df[df['ride_distance'] == 0]
print(f"\nFound {len(zero_dist_rides)} rides with a distance of 0 km.")
print("Booking status for these rides:\n", zero_dist_rides['booking_status'].value_counts())
# Observation: Most zero-distance rides are cancellations. This makes sense.
# They represent bookings that were cancelled before the trip started.
# For this analysis, we will keep them as they are valid records of cancellations.


# --- 6. Consolidate Cancellation Data ---
# To simplify analysis, we'll merge the two cancellation reason columns
# into a single, comprehensive 'cancellation_reason' column.
def get_cancellation_reason(row):
    if pd.notna(row['canceled_rides_by_customer']):
        return row['canceled_rides_by_customer']
    elif pd.notna(row['canceled_rides_by_driver']):
        return row['canceled_rides_by_driver']
    else:
        return 'Not Cancelled'

df['cancellation_reason'] = df.apply(get_cancellation_reason, axis=1)
# Now, we can drop the original, now redundant, cancellation columns.
df.drop(columns=['canceled_rides_by_customer', 'canceled_rides_by_driver'], inplace=True)
print("\n✅ Consolidated cancellation reasons into a single 'cancellation_reason' column.")
print("Unique values in new column:\n", df['cancellation_reason'].unique())


# --- 7. Final Inspection of the Cleaned DataFrame ---
# Let's do a final check of our work.
print("\n--- Final Technical Summary of Cleaned Data (df.info()) ---")
df.info()

print("\n--- First 5 Rows of Cleaned Data (df.head()) ---")
display(df.head(10))


# --- 8. Save the Cleaned Data ---
# This is the most important step. We now have a clean, reliable dataset
# to use for all future steps of the project (SQL, Power BI, etc.).
try:
    df.to_csv('../data/ola_data_cleaned.csv', index=False)
    print("\n✅ Successfully saved the cleaned data to 'data/ola_data_cleaned.csv'")
except Exception as e:
    print(f"\n❌ Error saving file: {e}")


✅ Raw dataset loaded successfully for cleaning.

Original column names:
 Index(['Date', 'Time', 'Booking_ID', 'Booking_Status', 'Customer_ID',
       'Vehicle_Type', 'Pickup_Location', 'Drop_Location', 'V_TAT', 'C_TAT',
       'Canceled_Rides_by_Customer', 'Canceled_Rides_by_Driver',
       'Incomplete_Rides', 'Incomplete_Rides_Reason', 'Booking_Value',
       'Payment_Method', 'Ride_Distance', 'Driver_Ratings', 'Customer_Rating',
       'Vehicle Images'],
      dtype='object')

Cleaned column names:
 Index(['date', 'time', 'booking_id', 'booking_status', 'customer_id',
       'vehicle_type', 'pickup_location', 'drop_location', 'v_tat', 'c_tat',
       'canceled_rides_by_customer', 'canceled_rides_by_driver',
       'incomplete_rides', 'incomplete_rides_reason', 'booking_value',
       'payment_method', 'ride_distance', 'driver_ratings', 'customer_rating',
       'vehicle_images'],
      dtype='object')

✅ Dropped unnecessary columns: ['time', 'vehicle_images']

✅ Renamed 'date' column

,booking_timestamp,booking_id,booking_status,customer_id,vehicle_type,pickup_location,drop_location,v_tat,c_tat,incomplete_rides,incomplete_rides_reason,booking_value,payment_method,ride_distance,driver_ratings,customer_rating,cancellation_reason
0,2024-07-26 14:00:00,CNR7153255142,Canceled by Driver,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,NaN,NaN,444,NaN,0,NaN,NaN,Personal & Car related issue
1,2024-07-25 22:20:00,CNR2940424040,Success,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,False,NaN,158,Cash,13,4.1,4.0,Not Cancelled
2,2024-07-30 19:59:00,CNR2982357879,Success,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,False,NaN,386,UPI,40,4.2,4.8,Not Cancelled
3,2024-07-22 03:15:00,CNR2395710036,Canceled by Customer,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,NaN,NaN,384,NaN,0,NaN,NaN,Driver is not moving towards pickup location
4,2024-07-02 09:02:00,CNR1797421769,Success,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,False,NaN,822,Credit Card,45,4.0,3.0,Not Cancelled
5,2024-07-13 04:42:00,CNR8787177882,Success,CID802429,Mini,Kadugodi,Vijayanagar,231.0,90.0,False,NaN,173,UPI,41,3.4,4.6,Not Cancelled
6,2024-07-23 09:51:00,CNR3612067560,Success,CID476071,Bike,Tumkur Road,Whitefield,133.0,40.0,False,NaN,140,Cash,49,3.2,4.5,Not Cancelled
7,2024-07-11 11:12:00,CNR5374902489,Canceled by Driver,CID735691,Prime Plus,Bannerghatta Road,Sarjapur Road,NaN,NaN,NaN,NaN,344,NaN,0,NaN,NaN,Personal & Car related issue
8,2024-07-01 19:19:00,CNR5030602354,Driver Not Found,CID999840,Mini,Chamarajpet,Peenya,NaN,NaN,NaN,NaN,839,NaN,0,NaN,NaN,Not Cancelled
9,2024-07-18 01:31:00,CNR6328453219,Canceled by Driver,CID907133,Auto,RT Nagar,Varthur,NaN,NaN,NaN,NaN,893,NaN,0,NaN,NaN,Personal & Car related issue



✅ Successfully saved the cleaned data to 'data/ola_data_cleaned.csv'


# Detailed Analysis of Your Day 2 Output

Let's break down what each section of the output means and the insights we've gained.

## Column Cleaning & Dropping

**What it shows**: The first two outputs confirm that our script successfully standardized all column names to lowercase with underscores and then removed the redundant `time` and `vehicle_images` columns.

**Analysis**: This is a foundational step in creating professional, readable code. Our DataFrame is now lighter and easier to work with. The final `df.info()` confirms we now have 17 columns, down from the original 20.

## incomplete_rides Conversion

**What it shows**: The output `Unique values in 'incomplete_rides': [nan 'No' 'Yes']` reveals that the column contained three distinct values. Our script then successfully converted the 'Yes'/'No' strings.

**Analysis**: In the final `df.info()` output, you'll notice `incomplete_rides` is still listed as an object type, not a boolean. This is expected and correct pandas behavior. When a column contains both boolean values (True/False) and missing values (NaN), pandas "upcasts" the data type to object so it can hold both. The underlying True/False values are still there and are perfectly usable for filtering and analysis.

## Anomaly Investigation (ride_distance = 0)

**What it shows**: We found a staggering **39,057 rides** with a distance of 0. The `value_counts()` immediately tells us why: these are all the rides that were not successful. They are Canceled by Driver, Canceled by Customer, or Driver Not Found.

**Analysis**: This is a critical insight. It confirms our hypothesis that 0-distance rides are not "errors" but are valid records of bookings that failed before the trip could start. Keeping them in our dataset is the correct decision. If we had removed them, we would have thrown away over a third of our data and lost the ability to analyze why these bookings failed.

## Cancellation Reason Consolidation

**What it shows**: The script successfully combined the two separate cancellation columns into one new column: `cancellation_reason`. The unique values show all the different reasons, plus the "Not Cancelled" category for successful rides.

**Analysis**: This is a major improvement for our future analysis. Instead of needing complex logic to check two different columns, we can now answer questions about cancellations by filtering a single, clean column. For example, `df[df['cancellation_reason'] == 'Driver asked to cancel']` is now a very simple operation.

## The Final Cleaned DataFrame

This is the "after" picture, showing the result of all our hard work.

### Key Data Quality Improvements

| Aspect | Before | After | Impact |
|--------|--------|-------|--------|
| Column names | Mixed case with spaces | Lowercase with underscores | Consistent, code-friendly |
| Column count | 20 columns | 17 columns | Streamlined, focused dataset |
| Date handling | `Date` column | `booking_timestamp` (datetime64) | Time-series analysis ready |
| Cancellation data | 2 separate columns | 1 consolidated column | Simplified analysis |
| Boolean data | String values | Proper boolean handling | Logic operations ready |

### Data Structure Insights

- **booking_timestamp**: Correctly set as a `datetime64[ns]` type, which is essential for any time-based analysis
- **No More Redundancy**: The `time` and `vehicle_images` columns are gone
- **Structural NaNs**: The `head()` output clearly shows the pattern we discovered. Successful rides (like index 1, 2, 4) have values for `v_tat`, `c_tat`, `payment_method`, etc. Unsuccessful rides (like index 0, 3) have NaN in these columns because those events never occurred. This confirms our understanding of the data's structure
- **Consolidated Data**: The last column, `cancellation_reason`, provides a clear, immediate explanation for the status of each ride

In summary, we have masterfully transformed a raw, slightly messy dataset into a clean, structured, and reliable asset. The data now makes logical sense, the data types are correct, and it is perfectly prepared for the next stage of our project.